In [1]:
from pyspark.sql import SparkSession

In [2]:
spark =SparkSession.builder.appName('Missing').getOrCreate()

In [5]:
training = spark.read.csv('Machine.csv', header=True,inferSchema=True)

In [15]:
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [14]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [ ]:
[Age,Experience]----> new feature--->independent feature

In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [18]:
output=featureassembler.transform(training)

In [19]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [20]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [24]:
finalized_data=output.select("Independent Features","Salary")

In [25]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [26]:
##train test split
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [27]:
### Coefficients
regressor.coefficients

DenseVector([-58.8235, 1666.6667])

In [28]:
### Intercepts
regressor.intercept

15196.078431372378

In [29]:
## Prediction
pred_results=regressor.evaluate(test_data)

In [30]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18784.313725490192|
|          [30.0,8.0]| 25000|26764.705882352937|
+--------------------+------+------------------+



In [32]:
pred_results.rootMeanSquaredError

1515.268947949933